Requirement: by Toon

1.) install dlib -- from here https://www.youtube.com/watch?v=AUJKdehF2ZA

2.) opencv has to be version 4.5.1.48

In [1]:
import dlib, cv2,os
import matplotlib.pyplot as plt
import numpy as np
from imutils.face_utils import FaceAligner

In [2]:
pose_predictor=dlib.shape_predictor('../FaceReco-master/FaceReco/shape_predictor_68_face_landmarks.dat')
fa = FaceAligner(pose_predictor)
face_encoder=dlib.face_recognition_model_v1('../FaceReco-master/FaceReco/dlib_face_recognition_resnet_model_v1.dat')
detector = dlib.get_frontal_face_detector()
modelFile = '../FaceReco-master/FaceReco/opencv_face_detector_uint8.pb'
configFile = '../FaceReco-master/FaceReco/opencv_face_detector.pbtxt'
net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

In [3]:
faces=[]
name=[]
trainpath = "train"

In [4]:
for im in os.listdir(trainpath):
    print(im)
    img = cv2.imread(os.path.join(trainpath,im))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    frameHeight = img.shape[0]
    frameWidth = img.shape[1]
    blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [104, 117,   123], False, False)
    net.setInput(blob)
    detections = net.forward()
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            faceAligned =fa.align(img, gray,dlib.rectangle(x1,y1,x2,y2))
            landmark = pose_predictor(faceAligned,dlib.rectangle(0,0,faceAligned.shape[0],faceAligned.shape[1]))
            face_descriptor = face_encoder.compute_face_descriptor(faceAligned, landmark, num_jitters=2)
            faces.append(face_descriptor)
            name.append(im)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'train'

In [ ]:
faces = np.array(faces)
name = np.array(name)
np.save('face_repr.npy', faces)
np.save('labels.npy', name)

In [ ]:
faces = np.load("face_repr.npy")
name = np.load("labels.npy")
# image = cv2.imread("../Test pytorch/Dataset/images/pins_Chris Evans/Chris Evans24_1196.jpg")
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
# frameHeight = image.shape[0]
# frameWidth = image.shape[1]
# blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104, 117, 123], False, False)
# net.setInput(blob)
# detections = net.forward()

In [ ]:
# scores=[]
# for i in range(detections.shape[2]):
#     confidence = detections[0, 0, i, 2]
#     if confidence > 0.7:
#         x1 = int(detections[0, 0, i, 3] * frameWidth)
#         y1 = int(detections[0, 0, i, 4] * frameHeight)
#         x2 = int(detections[0, 0, i, 5] * frameWidth)
#         y2 = int(detections[0, 0, i, 6] * frameHeight)
#         faceAligned = fa.align(image, gray,dlib.rectangle(x1,y1,x2,y2))
#         landmark = pose_predictor(faceAligned,dlib.rectangle(0,0,faceAligned.shape[0],faceAligned.shape[1]))
#         face_descriptor = face_encoder.compute_face_descriptor(faceAligned, landmark, num_jitters=2)
#         score = np.linalg.norm(faces - np.array(face_descriptor), axis=1)
#         scores.append(score)
#         imatches = np.argsort(score)
#         score = score[imatches]
#         print(name[imatches][:10].tolist(), score[:10].tolist())

In [ ]:
def predict(path):
    result_name = []    
    for file in os.listdir(path):
        image = cv2.imread(path +'/'+file)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        frameHeight = image.shape[0]
        frameWidth = image.shape[1]
        blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104, 117, 123], False, False)
        net.setInput(blob)
        detections = net.forward()
        scores=[]
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.7:
                x1 = int(detections[0, 0, i, 3] * frameWidth)
                y1 = int(detections[0, 0, i, 4] * frameHeight)
                x2 = int(detections[0, 0, i, 5] * frameWidth)
                y2 = int(detections[0, 0, i, 6] * frameHeight)
                faceAligned = fa.align(image, gray,dlib.rectangle(x1,y1,x2,y2))
                landmark = pose_predictor(faceAligned,dlib.rectangle(0,0,faceAligned.shape[0],faceAligned.shape[1]))
                face_descriptor = face_encoder.compute_face_descriptor(faceAligned, landmark, num_jitters=2)
                score = np.linalg.norm(faces - np.array(face_descriptor), axis=1)
                scores.append(score)
                imatches = np.argsort(score)
                score = score[imatches]
                a = name[imatches][:10].tolist()
                b = score[:10].tolist()
                #print(a,b)
                c = name[imatches][0].split(' ')
                result_name.append(c[0])
                #print(result_name)
    return result_name

In [ ]:
result_name = predict("test/")

['Bill Gates8_608.jpg', 'Johnny Depp20_1841.jpg', 'Mark Zuckerberg5_2063.jpg', 'elon musk11_1533.jpg', 'jeff bezos0_2040.jpg', 'Keanu Reeves12_1965.jpg', 'Chris Evans5_1235.jpg', 'Dwayne Johnson10_1558.jpg', 'Emma Watson32_1991.jpg', 'scarlett johansson15_3937.jpg'] [0.5587727405156723, 0.6159557417525423, 0.6347384751560347, 0.6505303915283986, 0.6843857762570372, 0.6982701215358881, 0.7181784988272977, 0.7422483717597987, 0.7638469569491508, 0.9453734287637487]
['Bill']
['Emma Watson32_1991.jpg', 'Bill Gates8_608.jpg', 'Mark Zuckerberg5_2063.jpg', 'Johnny Depp20_1841.jpg', 'elon musk11_1533.jpg', 'jeff bezos0_2040.jpg', 'Chris Evans5_1235.jpg', 'scarlett johansson15_3937.jpg', 'Keanu Reeves12_1965.jpg', 'Dwayne Johnson10_1558.jpg'] [0.6233227540058246, 0.6486979180517508, 0.6689461026116051, 0.7082404879156053, 0.7422376371921533, 0.7580543923890731, 0.7633715381310457, 0.7853989618408813, 0.7881108598296219, 0.8325254766334546]
['Bill', 'Emma']
['Bill Gates8_608.jpg', 'Dwayne Johnso

In [ ]:
# print(result_name)
# print(len(name_list))

In [ ]:
name_list = os.listdir("test")
name_list_split = [i.split(' ') for i in name_list]
print(name_list_split[2][0])

Bill


In [ ]:
correct = 0
all = len(name_list)
for i in range(len(name_list)):
    if result_name[i]==name_list_split[i][0]:
        correct = correct + 1

In [ ]:
print("Accuracy: ",correct*100/all,"%")

Accuracy:  93.0 %


## Test with Low Resolution images

In [ ]:
result_name = predict("test_LR/pins_Chris Evans")

['Keanu Reeves12_1965.jpg', 'Chris Evans5_1235.jpg', 'Johnny Depp20_1841.jpg', 'jeff bezos0_2040.jpg', 'Bill Gates8_608.jpg', 'elon musk11_1533.jpg', 'Dwayne Johnson10_1558.jpg', 'Mark Zuckerberg5_2063.jpg', 'Emma Watson32_1991.jpg', 'scarlett johansson15_3937.jpg'] [0.5880569212961307, 0.6141363602838806, 0.6482607708488015, 0.7136106089529406, 0.7241552797083549, 0.7501041050869502, 0.7570534010580506, 0.8081175291711697, 0.8446458110306755, 0.8978081789179361]
['Keanu']
['Keanu Reeves12_1965.jpg', 'Chris Evans5_1235.jpg', 'Dwayne Johnson10_1558.jpg', 'Johnny Depp20_1841.jpg', 'jeff bezos0_2040.jpg', 'elon musk11_1533.jpg', 'Mark Zuckerberg5_2063.jpg', 'Bill Gates8_608.jpg', 'Emma Watson32_1991.jpg', 'scarlett johansson15_3937.jpg'] [0.6385901371185561, 0.6731996676638603, 0.7098485138908056, 0.7257514648920957, 0.7343892710365694, 0.7381713955800422, 0.7768070712788547, 0.7880971845938529, 0.8739451804853503, 0.8942306374447062]
['Keanu', 'Keanu']
['Chris Evans5_1235.jpg', 'Johnny D

In [ ]:
name_list = os.listdir("test_LR/pins_Chris Evans")
name_list_split = [i.split(' ') for i in name_list]
print(name_list_split[2][0])

Chris


In [ ]:
correct = 0
all = len(name_list)
for i in range(len(name_list)):
    if result_name[i]==name_list_split[i][0]:
        correct = correct + 1

In [ ]:
print("Accuracy: ",correct*100/all,"%")

Accuracy:  80.0 %
